In [17]:
token = ''

lab_number = 10
phys_course = 141

if phys_course == 161:
    students_per_group = 4
    course_id = 381071 #161
    #161
    lab_names = [ 
            'Tutorial',
            'Introduction to Statistics and Data Analysis',
            'Motion with Uniform Acceleration',
            'Motion with Non-Uniform Acceleration',
            'Simple Harmonic Motion',
            'The Physical Pendulum',
            'Waves and Sound',
            'One Dimensional Collisions',
            'Two Dimensional Collisions',
            'The Rotating Bar',
            'The Gyroscopoe']

elif phys_course == 141:
    students_per_group = 4
    course_id = 395370   #141
    lab_names = [ 
        'Tutorial',
        'Introduction to Statistics and Data Analysis',
        'Computation of Kinematics',
        'Motion Due to Uniform Acceleration',
        'Free Fall and Air Drag',
        'Two Dimensional Motion with Drag',
        'One Dimensional Collisions',
        'Two Dimensional Collisiions',
        'The Inclined Plane',
        'The Gyroscope',
        'Gravitational Motion']
    
elif phys_course == 241:
    students_per_group = 4
    course_id = 398369 #241
    lab_names = ['Tutorial',
            'Electric Fields',
            'Electric Fields Computational Lab',
            'Capacitance',
            'DC Circuits',
            'Magnetic Fields and Magnetic Forces',
            'Motion of Charged Particles in Fields',
            'Charged Particles Computational Lab',
            'Faraday\'s Law',
            'AC Circuits',
            'AC Circuits Computational Lab']

elif phys_course == 261:
    students_per_group = 4
    course_id = 381076
    lab_names = ['Tutorial',
            'Electric Fields',
            'Electric Fields Computational Lab',
            'Capacitance',
            'DC Circuits',
            'Magnetic Fields and Magnetic Forces',
            'Motion of Charged Particles in Fields',
            'Charged Particles Computational Lab',
            'Faraday\'s Law',
            'AC Circuits',
            'AC Circuits Computational Lab']
    
#phys_course = 261
# you can get the course id from the course url, it's the last number in the below url
# https://umich.beta.instructure.com/courses/358386
#course_id = 395370   #141
#phys_course = 141
#course_id = 398369 #241
#phys_course = 241
#course_id = 381076
#phys_course = 261
# you can also use the code in the docstrings below to look up the course id instead of hard coding it
# the code just has to be placed below the
#141
#241/261

#141



#241/261




"""
# 161



#141



json = get('courses')
# print(json)
print( [x['course_code'] for x in json])
ids = [x['id'] for x in json if '136' in x['course_code']]
print(ids)
course_id = ids[0]
"""

# set the group_set_id to an integer to use that group set rather than making a new one
# set the group_set_id to None to create a new group set with the specified name (below)
# NOTE: This will not delete existing groups from the group set, though it WILL re-assign the students in them
#       ^ that behavior could be added (see delete(...) below), but it seems to me it's easier to just make new groups
# group_set_id = 21746
group_set_id = None


full_lab_name = 'Lab {:d}: {}'.format(lab_number, lab_names[lab_number])
group_set_name = full_lab_name

group_name_fmt = 'Lab {} {} - Group {}'

#########################################################################################
# Obviously you can edit the stuff below, but I tried to make it so you shouldn't have to
#########################################################################################

import requests
import sys
import math
# from pprint import pprint as print
import random
# random number generator for shuffling the groups
# r = random.SystemRandom()
r = random.Random() # we'll use the python random instead so we can seed it

def format_url(url, v1=True, rawURL=False):
    if rawURL:
        return url

    # assumed v1 specified by url
    if '/api/' in url:
        return 'https://umich.instructure.com' + url
    if 'api/' in url:
        return 'https://umich.instructure.com/' + url

    # canvasURL = 'https://umich.instructure.com/api/lti/'
    canvasURL = 'https://umich.instructure.com/api/lti/' # resets weekly, use .test. for monthly

    if v1:
        # canvasURL = 'https://umich.instructure.com/api/v1/'
        canvasURL = 'https://umich.instructure.com/api/v1/' # resets weekly

        # print('Raw url: ' + url)
    return canvasURL + url

def get(url, params=None, all=True, v1 = True, rawURL=False):
    canvasURL = format_url(url,v1,rawURL)
    response = {}

    if 'per_page' not in url:
        default_page_amount = 25
        if '?' in url:
            canvasURL += '&per_page=%d' % default_page_amount
        else:
            canvasURL += '?per_page=%d' % default_page_amount

    try:
        got = requests.get(canvasURL, params=params, headers={'Authorization': 'Bearer %s'%token})
        response = got.json()
        if 'errors' not in response:
            if all and 'next' in got.links:
                # print('Continuation: ' + str(got.links['next']))
                # print(got.links['next']['url'])
                # print(str(got.links))
                next = got.links['next']['url']
                response += get(next,params,all=True,v1=v1,rawURL=True)
            return response
    except Exception as e:
        print("Exceptions thrown in GET: " + str(e),file=sys.stderr)
        raise e
    print("Could not GET url: " + canvasURL,file=sys.stderr)
    if 'errors' in response:
        print("Error:",file=sys.stderr)
        print(response['errors'],file=sys.stderr)
    return response


def post(url, dictData=None, v1 = True, retry=5):
    canvasURL = format_url(url,v1,False)
    response = {}
    for attempt in range(1+int(retry)):
        if attempt > 0:
            print('Retrying [{}] POST {}'.format(attempt,canvasURL))
        try:
            req = requests.post(canvasURL,data=dictData, headers={'Authorization': 'Bearer %s'%token})
            response.update(req.json())
            if 'errors' not in response:
                return response
        except Exception as e:
            print("Exceptions thrown in POST: " + str(e),file=sys.stderr)
            response['python_exception_message'] = str(e)
            response['python_exception'] = e
        print("Could not POST url: " + canvasURL,file=sys.stderr)
        if 'errors' in response:
            print("Error:",file=sys.stderr)
            print(response['errors'],file=sys.stderr)
        else:
            response['errors'] = 'PYTHON-EXCEPTION'
    return response

def put(url, dictData=None, v1 = True, retry=5):
    canvasURL = format_url(url,v1,False)
    response = {}
    for attempt in range(1+int(retry)):
        if attempt > 0:
            print('Retrying [{}] PUT {}'.format(attempt,canvasURL))
        try:
            req = requests.put(canvasURL,data=dictData, headers={'Authorization': 'Bearer %s'%token})
            response.update(req.json())
            if 'errors' not in response:
                return response
        except Exception as e:
            print("Exceptions thrown in PUT: " + str(e),file=sys.stderr)
            response['python_exception_message'] = str(e)
            response['python_exception'] = e
        print("Could not PUT url: " + canvasURL,file=sys.stderr)
        if 'errors' in response:
            print("Error:",file=sys.stderr)
            print(response['errors'],file=sys.stderr)
        else:
            response['errors'] = 'PYTHON-EXCEPTION'

    return response

def delete(url, v1 = True, retry=5):
    canvasURL = format_url(url,v1,False)
    response = {}
    for attempt in range(1+int(retry)):
        if attempt > 0:
            print('Retrying [{}] DELETE {}'.format(attempt,canvasURL))
        try:
            req = requests.delete(canvasURL, headers={'Authorization': 'Bearer %s'%token})
            response.update(req.json())
            if 'errors' not in response:
                return response
        except Exception as e:
            print("Exceptions thrown in DELETE: " + str(e),file=sys.stderr)
        print("Could not DELETE url: " + canvasURL,sys.stderr)
        if 'errors' in response:
            print("Error:",file=sys.stderr)
            print(response['errors'],file=sys.stderr)
    return response

delete = lambda : print('DELETE has been disabled to prevent potential removal of unintended groups',sys.stderr)

def make_group_set(course_id, name, **kwargs):
    # the group set must not have a taken name, or there will be an error (400?) posting the group. The PUT api must be used in that case.
    # create_group_count=None, self_signup=None, auto_leader=None, group_limit=None
    defaults = {
        'name' : name,
        'create_group_count':None,
        'self_signup':None,
        'auto_leader':None,
        'group_limit':None
    }
    defaults.update(kwargs)
    return post('/api/v1/courses/{course_id}/group_categories'.format(course_id=course_id),
         dictData=defaults)

def delete_group_set(group_id):
    return delete('/api/v1/group_categories/{}'.format(group_id))


def get_group_sets(courseID):
    return get('courses/{course_id}/group_categories'.format(course_id=courseID))

def get_group_set_groups(groupSetID):
    return get('group_categories/{}/groups'.format(groupSetID))


def get_sections(course_id, include=['students']):
    return get('courses/{}/sections'.format(course_id),params=({'include':include} if include is not None else None))


def get_section_names(course_id):
    return list(map(lambda x : x['name'],get_sections(course_id,include=None)))

def get_section_students(section_id):
    return get('sections/{}'.format(section_id),params={'include':'students'})

def get_students_by_section(course_id):
    sects = get_sections(course_id,include=['students'])
    out = {}
    for sect in sects:
        l = []
        for s in sect['students']:
            l.append(s['id'])
        out[sect['id']] = l
    return out

def make_group(group_set_id, name, **kwargs):
    defaults = {
      'name': name,
      'description': None,
      'is_public': False,
      'join_level': 'invitation_only',
    }
    defaults.update(kwargs)
    url = 'group_categories/{}/groups'.format(group_set_id)
    # if group_set_id is None:
    #     url = 'groups'
    return post(url,dictData=defaults)

def set_members_for_group(group_id, user_ids):
    # for user_id in user_ids:
    #     post('groups/{}/memberships'.format(group_id),dictData={'user_id':str(user_id)})
    # return None
    id_str = user_ids
    return put('groups/{}'.format(group_id),dictData={'members[]':id_str})

    # return requests.put('http://httpbin.org/anything',data={'members[]':id_str}).json()

def form_groups_in_group_set(course_id, group_set_id):
    sections = get_sections(course_id)
    trim_section_name = lambda x : x[x.find('Section'):] if 'Section' in x else x
    section_names = dict(map(lambda x : (x['id'],trim_section_name(x['name'])),sections))

    total_group_count = 0

    section_student_ids = {}
    #print(sections)
    for sect in sections:
        #print(sect)
        l = []
        try:
            for s in sect['students']:
                l.append(s['id'])
            section_student_ids[sect['id']] = l
        except:
            print('exception on:\n')
            print(sect)
            
    for section_id in section_names:
        try:
            current_students = section_student_ids[section_id]
            r.seed(a=3**lab_number+lab_number+256*section_id)
            r.shuffle(current_students)

            # I was going to go for the fixed group count. Instead, I'll just evenly
            #    distribute the pad slots from making the last group
            # how many students to pad in the groups
            # pad_count = group_count*students_per_group - len(current_students)

            ending_group_size = len(current_students) % students_per_group
            pad_students = 0 if ending_group_size == 0 else (students_per_group - ending_group_size)
            group_count = math.ceil(len(current_students) / students_per_group)

            pad_groups = r.sample(range(group_count),pad_students)


            for i in range(group_count):
                group_size = students_per_group
                # This randomly assigns the smaller sized groups
                if i in pad_groups:
                    group_size -= 1
                # If we want them at the beginning, I think `if i < pad_students:` would work
                group = current_students[:group_size]
                current_students = current_students[group_size:]
                group_name = group_name_fmt.format(lab_number, section_names[section_id], i+1)
                g = make_group(group_set_id,group_name)
                gid = g['id']
                added = set_members_for_group(gid, group)
            print("Added section {}'s {} groups".format(section_names[section_id],group_count))
            total_group_count += group_count
        except: None
            
    return total_group_count


In [18]:

import pandas as pd

#def zoom_breakout_rooms(section,group_set)
    

# Code to find the course by name rather than using a given id
# json = get('courses')
# print(json)
# print( [x['course_code'] for x in json])
# ids = [x['id'] for x in json if '136' in x['course_code']]
# print(ids)
# i = ids[0]
# course_id = i

if course_id is None:
    print('We must have a course id to add the group set to it.')
    print('The course id can be obtained from the URL for the course page on Canvas')
    print('  https://umich.beta.instructure.com/courses/<COURSE ID>')
    exit()

groupsets = get_group_sets(course_id)
gs_names = list(map(lambda x : x['name'], groupsets))
if group_set_id is None:

    if group_set_name in gs_names:
        conflict_id = ', '.join([str(x['id']) for x in groupsets if group_set_name == x['name']])
        print('We cannot create the group set with name {}.'.format(group_set_name))
        print('The group already exists with id={}'.format(conflict_id))
        print('Specify the group_set_id in the script to avoid this error, or choose a different group_set_name')
        exit()

    print('Making Group Set: ' + group_set_name)
    group_set = make_group_set(course_id, group_set_name)

    group_set_id = group_set['id']
    print('Created Group Set: %d'%group_set_id)

# if we wanted to delete the group that existed (but this would be much better done through canvas api
# gs = list(map(lambda x : (x['id'],x['name']),groupsets))
# for i,name in gs:
#     if name == 'Testing automated groups - 2':
#         delete_group_set(i)

print('Forming Groups')
count = form_groups_in_group_set(course_id, group_set_id)
print('Finished adding {} groups to Group Set "{}" [{}]'.format(count,group_set_name,group_set_id))
print("It's url might(?) be: https://umich.beta.instructure.com/courses/{}/groups#tab-{}".format(course_id,group_set_id))
#exit()




Making Group Set: Lab 10: Gravitational Motion
Created Group Set: 29134
Forming Groups
Added section Section 010's 5 groups
Added section Section 011's 5 groups
Added section Section 012's 5 groups
Added section Section 013's 5 groups
Added section Section 014's 5 groups
Added section Section 015's 5 groups
Added section Section 020's 5 groups
Added section Section 021's 5 groups
Added section Section 022's 5 groups
Added section Section 023's 5 groups
Added section Section 024's 5 groups
Added section Section 025's 5 groups
Added section Section 030's 5 groups
Added section Section 031's 5 groups
Added section Section 032's 5 groups
Added section Section 033's 5 groups
Added section Section 040's 5 groups
Added section Section 041's 5 groups
Added section Section 042's 5 groups
Added section Section 043's 5 groups
Added section Section 071's 5 groups
Added section Section 072's 5 groups
Added section Section 073's 5 groups
Added section Section 074's 5 groups
Added section Section 075

In [19]:

import pandas as pd
import os

groupsets = get_group_sets(course_id)

for group in groupsets:
    #print(group['name'])
    try:        
        if int(group['name'][4]) == lab_number:
            #print(group['name'])
            group_ids = get_group_set_groups(group['id'])
            print('Recovered group ids for output to zoom .csv')
        #print(int(group['name'][4:6]))
        elif int(group['name'][4:6]) == lab_number:
            #print(group['name'])
            group_ids = get_group_set_groups(group['id'])
            print('Recovered group ids for output to zoom .csv')
    except: None
        
df = pd.DataFrame(data=None,columns=['Pre-assign Room Name','Email Address'])

if phys_course == 261: cap_num = 4
else: cap_num = 5

for grp in group_ids:
    this_name = grp['name']
    sec = this_name.find('Section')+8
    grp_num = this_name.find('Group')+6
    section = this_name[sec:sec+3]
    group_num = this_name[grp_num]
    for student in get('groups/{}/users'.format(grp['id'])):
        zs = student['login_id']+'@umich.edu'
        zb_series = {'Pre-assign Room Name': this_name, 'Email Address':zs}
        df = df.append(zb_series,ignore_index=True)
    if int(group_num) == cap_num:
        if not os.path.exists('./Zoom Rooms/'):
            os.mkdir('./Zoom Rooms/')
        if not os.path.exists('./Zoom Rooms/Lab {}/'.format(lab_number)):
            os.mkdir('./Zoom Rooms/Lab {}/'.format(lab_number))
        if not os.path.exists('./Zoom Rooms/Lab {}/Physics {}/'.format(lab_number,phys_course)):
            os.mkdir('./Zoom Rooms/Lab {}/Physics {}/'.format(lab_number,phys_course))
        print('Output {} groups to section {} lab {}'.format(group_num,section,lab_number))
        df.to_csv('./Zoom Rooms/Lab {}/Physics {}/phys{}_section{}_lab{}.csv'.format(lab_number,phys_course,phys_course,section,lab_number),index=False)
        df = pd.DataFrame(data=None,columns=['Pre-assign Room Name','Email Address'])
print('Done.')


Recovered group ids for output to zoom .csv
Output 5 groups to section 010 lab 10
Output 5 groups to section 011 lab 10
Output 5 groups to section 012 lab 10
Output 5 groups to section 013 lab 10
Output 5 groups to section 014 lab 10
Output 5 groups to section 015 lab 10
Output 5 groups to section 020 lab 10
Output 5 groups to section 021 lab 10
Output 5 groups to section 022 lab 10
Output 5 groups to section 023 lab 10
Output 5 groups to section 024 lab 10
Output 5 groups to section 025 lab 10
Output 5 groups to section 030 lab 10
Output 5 groups to section 031 lab 10
Output 5 groups to section 032 lab 10
Output 5 groups to section 033 lab 10
Output 5 groups to section 040 lab 10
Output 5 groups to section 041 lab 10
Output 5 groups to section 042 lab 10
Output 5 groups to section 043 lab 10
Output 5 groups to section 071 lab 10
Output 5 groups to section 072 lab 10
Output 5 groups to section 073 lab 10
Output 5 groups to section 074 lab 10
Output 5 groups to section 075 lab 10
Output